# Ejen AI Azure dengan Sokongan Protokol Konteks Model (MCP)

Notebook ini menunjukkan cara menggunakan Ejen AI Azure dengan alat Protokol Konteks Model (MCP) untuk mencipta ejen pintar yang boleh memanfaatkan pelayan MCP luaran bagi meningkatkan keupayaan.


## Pasang Pakej NuGet yang Diperlukan

Pertama, kita perlu memasang pakej Azure AI Agents Persistent yang menyediakan fungsi teras untuk bekerja dengan Azure AI Agents.


## Kelebihan Pengesahan Tanpa Kunci

Notebook ini menunjukkan **pengesahan tanpa kunci** yang memberikan beberapa kelebihan:
- ✅ **Tiada kunci API untuk diuruskan** - Menggunakan pengesahan berasaskan identiti Azure
- ✅ **Keselamatan dipertingkatkan** - Tiada rahsia disimpan dalam kod atau konfigurasi
- ✅ **Putaran kelayakan automatik** - Azure menguruskan kitaran hayat kelayakan
- ✅ **Akses berasaskan peranan** - Menggunakan Azure RBAC untuk kebenaran yang lebih terperinci

`DefaultAzureCredential` secara automatik akan menggunakan sumber kelayakan terbaik yang tersedia:
1. Identiti Terurus (apabila dijalankan di Azure)
2. Kelayakan Azure CLI (semasa pembangunan)
3. Kelayakan Visual Studio
4. Pembolehubah persekitaran (jika dikonfigurasikan)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Pasang pakej Azure Identity untuk pengesahan dengan perkhidmatan Azure menggunakan DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Import Nama Ruang yang Diperlukan

Import nama ruang yang diperlukan untuk Azure AI Agents dan Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Konfigurasi Azure AI Agent Client (Pengesahan Tanpa Kunci)

Tetapkan pembolehubah konfigurasi dan cipta PersistentAgentsClient menggunakan **pengesahan tanpa kunci**:
- **projectEndpoint**: Titik akhir projek Azure AI Foundry
- **modelDeploymentName**: Nama model AI yang telah dilancarkan (GPT-4.1 nano)
- **mcpServerUrl**: URL pelayan MCP (Microsoft Learn API)
- **mcpServerLabel**: Label untuk mengenal pasti pelayan MCP
- **DefaultAzureCredential**: Menggunakan identiti terurus, Azure CLI, atau sumber kelayakan lain (tidak memerlukan kunci API)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Buat Definisi Alat MCP

Buat definisi alat MCP yang berhubung dengan pelayan MCP Microsoft Learn. Ini akan membolehkan ejen mengakses kandungan dan dokumentasi Microsoft Learn.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Cipta Ejen AI

Cipta ejen AI yang berterusan dengan model dan alat MCP yang ditentukan. Ejen ini dikonfigurasi dengan:
- Model GPT-4.1 nano
- Arahan untuk menggunakan alat MCP sebagai bantuan
- Akses kepada pelayan Microsoft Learn MCP


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Buat Thread dan Hantar Mesej

Buat thread perbualan dan hantar mesej kepada pengguna yang bertanya tentang perbezaan antara Azure OpenAI dan OpenAI. Ini akan menguji keupayaan ejen untuk menggunakan alat MCP bagi memberikan maklumat yang tepat.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Konfigurasi Sumber Alat MCP (Tanpa Kunci)

Tetapkan sumber alat MCP. Untuk pendekatan benar-benar tanpa kunci, anda boleh menghapuskan header tersuai jika pelayan MCP menyokong pengesahan berasaskan identiti Azure. Contoh di bawah menunjukkan cara menambah header jika diperlukan, tetapi untuk senario tanpa kunci, ini mungkin tidak diperlukan.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Mulakan Larian Ejen

Cipta dan mulakan larian untuk memproses mesej pengguna. Ejen akan menggunakan alat dan sumber MCP yang telah dikonfigurasi untuk menghasilkan respons.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Pantau Jalankan dan Urus Kelulusan Alat (Tanpa Kunci)

Pantau status pelaksanaan ejen dan urus sebarang kelulusan alat yang diperlukan. Gelung ini:
1. Menunggu pelaksanaan selesai atau memerlukan tindakan
2. Meluluskan panggilan alat MCP secara automatik apabila diperlukan
3. Untuk pengesahan tanpa kunci, header mungkin tidak diperlukan jika pelayan MCP menyokong identiti Azure


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Paparkan Hasil Perbualan

Dapatkan dan paparkan semua mesej dalam urutan perbualan, menunjukkan soalan pengguna dan respons ejen. Mesej-mesej dipaparkan mengikut susunan kronologi dengan penunjuk masa dan peranan.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk memastikan ketepatan, sila ambil perhatian bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat yang kritikal, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
